<a href="https://colab.research.google.com/github/ruthetum/handson-ml-std/blob/master/ch15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter15 RNN과 CNN을 사용해 시퀀스 처리하기

In [2]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

import numpy as np
import os
from pathlib import Path

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

### 시계열 생성 함수 (단변량 시계열)

batch size 랑 step 길이를 매개변수로 입력

[batch_size, n_step, 1] 넘파이 배열로 출력



In [3]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # sine wave
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # sine wave
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # 잡음 추가
    return series[..., np.newaxis].astype(np.float32)

In [4]:
# 단변량 시계열 생성 확인
test = generate_time_series(5, 3)
test

array([[[ 0.15254667],
        [ 0.14995728],
        [ 0.19618355]],

       [[ 0.23656414],
        [ 0.35126457],
        [ 0.28141645]],

       [[-0.01466727],
        [ 0.35243037],
        [-0.28331563]],

       [[-0.15250996],
        [ 0.497087  ],
        [ 0.27007994]],

       [[ 0.2533832 ],
        [ 0.47112006],
        [ 0.19195215]]], dtype=float32)

### 훈련, 검증, 테스트 세트 생성

In [5]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

### 기준 성능 측정
- native forecasting : 시계열 마지막 값을 그대로 예측
- 완전 연결 네트워크

In [6]:
# native forecating
y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.020182354

In [7]:
# 완전 연결 네트워크
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

In [8]:
# 측정
model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0942 - val_loss: 0.0418
Epoch 2/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0266 - val_loss: 0.0173
Epoch 3/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0138 - val_loss: 0.0112
Epoch 4/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0097 - val_loss: 0.0087
Epoch 5/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0080 - val_loss: 0.0076
Epoch 6/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0069 - val_loss: 0.0068
Epoch 7/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0062 - val_loss: 0.0061
Epoch 8/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0056 - val_loss: 0.0055
Epoch 9/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0051 - val_loss: 0.0051
Epoch 10/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0048 - val_loss: 0.0047

In [9]:
model.evaluate(X_valid, y_valid)

63/63 [==============================] - 0s 1ms/step - loss: 0.0035


0.00354241905733943

### 간단한 RNN 구현

In [10]:
# 뉴런 1개
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

# step 마다 출력을 반환할려면 return_suquences=True 설정
# model = keras.models.Sequential([
#     keras.layers.SimpleRNN(1, return_sequences=True, input_shape=[None, 1])
# ])

In [11]:
model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 7s 34ms/step - loss: 0.1167 - val_loss: 0.1053
Epoch 2/20
219/219 [==============================] - 7s 32ms/step - loss: 0.0887 - val_loss: 0.0785
Epoch 3/20
219/219 [==============================] - 7s 32ms/step - loss: 0.0643 - val_loss: 0.0540
Epoch 4/20
219/219 [==============================] - 7s 32ms/step - loss: 0.0424 - val_loss: 0.0338
Epoch 5/20
219/219 [==============================] - 7s 30ms/step - loss: 0.0266 - val_loss: 0.0214
Epoch 6/20
219/219 [==============================] - 7s 31ms/step - loss: 0.0189 - val_loss: 0.0168
Epoch 7/20
219/219 [==============================] - 7s 33ms/step - loss: 0.0165 - val_loss: 0.0154
Epoch 8/20
219/219 [==============================] - 7s 32ms/step - loss: 0.0156 - val_loss: 0.0147
Epoch 9/20
219/219 [==============================] - 7s 30ms/step - loss: 0.0149 - val_loss: 0.0140
Epoch 10/20
219/219 [==============================] - 7s 31ms/step - loss: 0.0143 - val_lo

In [12]:
model.evaluate(X_valid, y_valid)

63/63 [==============================] - 0s 4ms/step - loss: 0.0108


0.010821439325809479

### 심층 RNN

In [13]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True,),
    keras.layers.SimpleRNN(1)
])

In [14]:
model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 20s 90ms/step - loss: 0.0203 - val_loss: 0.0048
Epoch 2/20
219/219 [==============================] - 19s 87ms/step - loss: 0.0040 - val_loss: 0.0035
Epoch 3/20
219/219 [==============================] - 19s 87ms/step - loss: 0.0033 - val_loss: 0.0033
Epoch 4/20
219/219 [==============================] - 19s 87ms/step - loss: 0.0031 - val_loss: 0.0032
Epoch 5/20
219/219 [==============================] - 19s 87ms/step - loss: 0.0031 - val_loss: 0.0031
Epoch 6/20
219/219 [==============================] - 19s 87ms/step - loss: 0.0030 - val_loss: 0.0032
Epoch 7/20
219/219 [==============================] - 20s 90ms/step - loss: 0.0030 - val_loss: 0.0030
Epoch 8/20
219/219 [==============================] - 20s 89ms/step - loss: 0.0029 - val_loss: 0.0031
Epoch 9/20
219/219 [==============================] - 19s 88ms/step - loss: 0.0030 - val_loss: 0.0028
Epoch 10/20
219/219 [==============================] - 20s 92ms/step - loss: 0.002

In [15]:
model.evaluate(X_valid, y_valid)

63/63 [==============================] - 1s 10ms/step - loss: 0.0026


0.0026042431127279997

### 출력층을 keras.layers.Dense(1)로 변경
- 이렇게 하면 출력층의 활성화 함수를 변경 가능
- SimpleRNN에서도 활성화 함수를 선택할 수 있지만 잘 동작하지 않음

In [16]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

model.evaluate(X_valid, y_valid)

Epoch 1/20
219/219 [==============================] - 14s 62ms/step - loss: 0.0407 - val_loss: 0.0074
Epoch 2/20
219/219 [==============================] - 14s 62ms/step - loss: 0.0053 - val_loss: 0.0045
Epoch 3/20
219/219 [==============================] - 13s 58ms/step - loss: 0.0041 - val_loss: 0.0039
Epoch 4/20
219/219 [==============================] - 13s 60ms/step - loss: 0.0036 - val_loss: 0.0035
Epoch 5/20
219/219 [==============================] - 14s 64ms/step - loss: 0.0033 - val_loss: 0.0037
Epoch 6/20
219/219 [==============================] - 13s 60ms/step - loss: 0.0032 - val_loss: 0.0034
Epoch 7/20
219/219 [==============================] - 14s 63ms/step - loss: 0.0031 - val_loss: 0.0033
Epoch 8/20
219/219 [==============================] - 13s 62ms/step - loss: 0.0031 - val_loss: 0.0029
Epoch 9/20
219/219 [==============================] - 13s 60ms/step - loss: 0.0030 - val_loss: 0.0031
Epoch 10/20
219/219 [==============================] - 14s 63ms/step - loss: 0.003

0.0026274442207068205

### 여러 타임 스텝 앞 예측

방법 1 : 한 스텝에 1개씩 10개 예측
- 이미 훈련된 모델을 사용하여 다음 값을 예측
- 해당 값을 입력으로 추가 (예측값을 실제 값처럼 사용)

In [17]:
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)

Y_pred = X[:, n_steps:]

방법2 : RNN으로 다음 값 10개를 한 번에 예측

In [18]:
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [19]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

Y_pred = model.predict(X_new)

In [20]:
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

In [21]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [22]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])
    
optimizer=keras.optimizers.Adam(lr=0.01)

In [23]:
model.compile(loss="mse", optimizer=optimizer, metrics=[last_time_step_mse])

### Layer Normalization

In [29]:
class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units, activation=None)
        self.layer_norm = LayerNormalization() # Layer Normalization
        self.activation = keras.activations.get(activation)
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [30]:
from tensorflow.keras.layers import LayerNormalization
model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True, input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

### LSTM Cell

In [31]:
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [32]:
# 이렇게 매개변수 형식으로 지정할 수도 있다.
# model = keras.models.Sequential([
#     keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences=True, input_shape=[None, 1]),
#     keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences=True),
#     keras.layers.TimeDistributed(keras.layers.Dense(10))
# ])

### GRU Cell

In [35]:
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid", input_shape=[None, 1]),                             
    keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train[:, 3::2], epochs=20, validation_data=(X_valid, Y_valid[:, 3::2]))

Epoch 1/20
219/219 [==============================] - 2s 8ms/step - loss: 0.0682 - last_time_step_mse: 0.0609 - val_loss: 0.0459 - val_last_time_step_mse: 0.0378
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0392 - last_time_step_mse: 0.0316 - val_loss: 0.0354 - val_last_time_step_mse: 0.0277
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0328 - last_time_step_mse: 0.0241 - val_loss: 0.0303 - val_last_time_step_mse: 0.0210
Epoch 4/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0286 - last_time_step_mse: 0.0186 - val_loss: 0.0273 - val_last_time_step_mse: 0.0175
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0259 - last_time_step_mse: 0.0151 - val_loss: 0.0250 - val_last_time_step_mse: 0.0143
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0242 - last_time_step_mse: 0.0132 - val_loss: 0.0236 - val_last_time_step_mse: 0.0125
Epoch 7/20
219/219 [========

### WaveNet

In [36]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))
for rate in (1, 2, 4, 8) * 2:
    model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal", activation="relu", dilation_rate=rate))
model.add(keras.layers.Conv1D(filters=10, kernel_size=1))
model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

Epoch 1/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0661 - last_time_step_mse: 0.0543 - val_loss: 0.0356 - val_last_time_step_mse: 0.0218
Epoch 2/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0322 - last_time_step_mse: 0.0183 - val_loss: 0.0300 - val_last_time_step_mse: 0.0171
Epoch 3/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0287 - last_time_step_mse: 0.0156 - val_loss: 0.0276 - val_last_time_step_mse: 0.0155
Epoch 4/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0266 - last_time_step_mse: 0.0140 - val_loss: 0.0261 - val_last_time_step_mse: 0.0139
Epoch 5/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0253 - last_time_step_mse: 0.0128 - val_loss: 0.0248 - val_last_time_step_mse: 0.0126
Epoch 6/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0241 - last_time_step_mse: 0.0118 - val_loss: 0.0239 - val_last_time_step_mse: 0.0121
Epoch 7/20
219/219 [========